In [2]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly-express kaleido
from fpdf import FPDF  # pip install fpdf

C:\Users\Admin\AppData\Local\Temp\ipykernel_1864\3387221087.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd  # pip install pandas


In [3]:
# Define the plotly template. Some other examples:
# "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "presentation"

In [4]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

In [5]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [6]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [7]:
# Print the DataFrame
print(df)

      sale_date  total_sales
0    2023-01-01          350
1    2023-01-02         1050
2    2023-01-03          300
3    2023-01-05          350
4    2023-01-06          630
..          ...          ...
335  2023-12-26         1100
336  2023-12-28          625
337  2023-12-29         1675
338  2023-12-30         1150
339  2023-12-31          840

[340 rows x 2 columns]


In [8]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    340 non-null    object
 1   total_sales  340 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [9]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    340 non-null    datetime64[ns]
 1   total_sales  340 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.4 KB


In [10]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2023-01-01,350
2023-01-02,1050
2023-01-03,300


In [11]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

C:\Users\Admin\AppData\Local\Temp\ipykernel_1864\3440663402.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df.resample('M').sum()


,total_sales
sale_date,
2023-01-31,28995
2023-02-28,18040
2023-03-31,22925
2023-04-30,24605
2023-05-31,21590
2023-06-30,16620
2023-07-31,25675
2023-08-31,21755
2023-09-30,19575


In [12]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2023-01-31,28995,Jan
2023-02-28,18040,Feb
2023-03-31,22925,Mar
2023-04-30,24605,Apr
2023-05-31,21590,May
2023-06-30,16620,Jun
2023-07-31,25675,Jul
2023-08-31,21755,Aug
2023-09-30,19575,Sep


In [13]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
# fig.write_image(output_dir / 'monthly_sales.png',
#                 width=1200,
#                 height=400,
#                 scale=4)

In [14]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [15]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# # Save the chart as a PNG image
# fig.write_image(output_dir / 'customer_sales.png',
#                 width=1200,
#                 height=400,
#                 scale=4)